In [1]:
import urllib.request
import urllib

dest = "C:/Users/Braeden Kuether/Desktop/template_test/"

fp = urllib.request.urlopen("https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000002488&type=10-K&dateb=&owner=exclude&count=10&search_text=")
mybytes = fp.read()
    
mystr = mybytes.decode("utf8")
fp.close()

In [3]:
urlBeg = "https://www.sec.gov/Archives/edgar/data/2488/"
urlEnd = "/Financial_Report.xlsx"

position = 0
strLen = 150
length = len(mystr)
years = 9
files = []

def findSlash(tempstr):
    return tempstr.find('/')

for i in range(years):
    tempstr = mystr[position:length]
    tenK = tempstr.find('<td nowrap="nowrap">10-K')
    if tenK != -1:
        tenK += position
        position = (tenK + strLen)
        url = mystr[tenK:position]
        
        for j in range(6):
            nextSlash = findSlash(url) + 1
            url = url[nextSlash:strLen]
            
        nextSlash = findSlash(url)
        url = url[0:nextSlash]
        url = urlBeg + url + urlEnd
        files.append(url)
    
#https://www.sec.gov/Archives/edgar/data/2488/000162828021001185/Financial_Report.xlsx        

In [5]:
k = 0
for f in files:
    k += 1
    urllib.request.urlretrieve(files[1], dest + "10-K" + str(k) + ".xlsx")   